In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import os

Using TensorFlow backend.


In [2]:
# return a map with the labels
def make_id_label_map():
    path = os.path.join('dataset', 'images')
    labels = dict()
    for i,value in enumerate(os.listdir(path)):
        labels[value] = i
        
    return labels

In [3]:
# create a .cvs file that contains the images urls and labels
def make_data_file():
    labels = make_id_label_map()
    
    path = os.path.join('dataset','images')
    path_file = os.path.join('dataset','data.csv')
    
    text_file = open(path_file, 'w')
    text_file.write('images,labels\n')

    for folder in os.listdir(path):
        folder_path = os.path.join(path,folder)
        for i in os.listdir(folder_path):
            text_file.write(os.path.join(folder_path,i)+','+str(labels[folder])+'\n')

    text_file.close()

In [4]:
def make_datagenerator():
    file_path = os.path.join('dataset','data.csv')
    df = pd.read_csv(file_path, header=0)
    #df = df.sample(frac=1).reset_index(drop=True)
    datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_dataframe(
        dataframe = df,
        x_col = 'images',
        y_col = 'labels',
        target_size = (224,224),
        batch_size = 3,
        class_mode = 'other'
    )
    return generator

In [5]:
model = tf.keras.applications.MobileNet(include_top=False, weights='imagenet',input_shape=(224,224,3))
model.trainable = False
model = tf.keras.Sequential([
    model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=10, activation='softmax')
])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                10250     
Total params: 3,239,114
Trainable params: 10,250
Non-trainable params: 3,228,864
_________________________________________________________________


In [10]:
#generator = make_datagenerator()
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(generator, steps_per_epoch=270, epochs=10)

Epoch 1/10
27/27 [==============================] - 50s 2s/step - loss: 2.4342 - acc: 0.0750
Epoch 2/10
27/27 [==============================] - 49s 2s/step - loss: 2.3534 - acc: 0.1184
Epoch 3/10
27/27 [==============================] - 49s 2s/step - loss: 2.3517 - acc: 0.0973
Epoch 4/10
27/27 [==============================] - 51s 2s/step - loss: 2.3558 - acc: 0.0926
Epoch 5/10
27/27 [==============================] - 48s 2s/step - loss: 2.3919 - acc: 0.1008
Epoch 6/10
27/27 [==============================] - 48s 2s/step - loss: 2.3630 - acc: 0.0891
Epoch 7/10
27/27 [==============================] - 50s 2s/step - loss: 2.3565 - acc: 0.1102
Epoch 8/10
27/27 [==============================] - 48s 2s/step - loss: 2.3766 - acc: 0.1067
Epoch 9/10
27/27 [==============================] - 48s 2s/step - loss: 2.3672 - acc: 0.0985
Epoch 10/10
27/27 [==============================] - 48s 2s/step - loss: 2.4075 - acc: 0.0996


In [9]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_directory(
        'dataset/images',
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse')

Found 853 images belonging to 10 classes.
